# Homework 3 


_Author:_ Muhammed Fatih Öztel

_Student ID:_ 150119907


In [1]:
import numpy as np

## Gradient Descent

In [2]:
learning_rate = 0.1
def error_surface(u, v): # error surface function that give in question
    return ((u* (np.exp(v) ) )-( 2 * v * (np.exp(-u))))**2 
     
def partial_u(u, v): # partial derivative u of error surface on 
    return 2 * (np.exp(v) + 2*v*(np.exp(-u))) * ( u*(np.exp(v)) - 2 * v *(np.exp(-u)))
def partial_v(u, v): # partial derivative u of error surface on 
    return 2 * ( u * ( np.exp(v)) - 2* (np.exp(-u)) )  * ( u*(np.exp(v)) - 2 * v *(np.exp(-u)))

### Question 5

In [3]:
u, v = 1, 1
e = error_surface(u, v) # initial error surface
counter = 0
while e >= 1e-14: # get out when smaller than 1e-14
    u,v = u - learning_rate * partial_u(u, v), v- learning_rate *partial_v(u, v) # updatingu and v -> gradient descent formula
    e =  error_surface(u, v) # find new error with new u and v
    counter += 1
counter #answer is 10 D

10

### Question 6

In [4]:
u, v # which is very close to E

(0.04473629039778207, 0.023958714099141746)

### Question 7

In [5]:
u, v = 1, 1
e = error_surface(u, v)
counter=  0
while counter<15: # for 15 iteration
    new_u = u - learning_rate * partial_u(u, v) # first update and get new u 
    u = new_u
    new_v = v - learning_rate * partial_v(u, v)  # update v with new u
    v = new_v
    e =  error_surface(u, v) # find error surface
    counter+=1 
e # answer is close to 10^-1 which is A

0.13981379199615315

## Logistic Regression

### Question 8

In [6]:
%%time
def target_line(): # targe tine function that generates 2 points 
    target = np.random.uniform(-1,1, size=4).reshape(2,2)
    return target
def find_boundary(p_x,p_y, line): # function to find boundary of points
    # formula to find is point above or belove the target function
    det = (line[0][1] - line[0][0]) * (p_y - line[1][0]) - (line[1][1]-line[1][0]) * (p_x-line[0][0]) 
    return 1 if det > 0 else -1


eout_list = [] # list of eouts
epoch_list =[] # number of epochs for every iteration
for i in range(100):
    dataset = np.insert(np.random.uniform(-1,1, size=100*2).reshape(100,2),0,1,axis=1) # generate random train dataset
    plane = target_line() # target line 
    wt_1 = np.zeros(3) # weight for previos epoch first intialize to zero
    sign_list=[] # sign list for train data
    sign_indexes = np.arange(len(dataset)) # list to represent dataset as indicies
    for i in range(len(dataset)): # addi signs for every point
        sign_list.append(find_boundary(dataset[i][1],dataset[i][2],plane))


    test_set = np.insert(np.random.uniform(-1,1, size=100*2).reshape(100,2),0,1,axis=1) # generate random test set
    test_sign_list=[] # test sign list
    for i in range(len(test_set)): #
        test_sign_list.append(find_boundary(test_set[i][1],test_set[i][2],plane))

    e_c = 0; # epoch counter
    weight_e = np.zeros(3) #weight for current epoch first intialize zero
    while True:
        np.random.shuffle(sign_indexes) # shuffle randomly the indices 
        for i in range(len(sign_indexes)): # update weight_e with every data point 
            yn = sign_list[sign_indexes[i]] # label of that point
            xn = dataset[sign_indexes[i]] # point itself
            err_in = - (yn * xn) / (1 + np.exp( yn * np.dot(weight_e,xn) )) # finding err in from formula
            weight_e = weight_e - 0.01 * err_in # find new weight 
        e_c += 1 # increment counter
        if  np.linalg.norm(wt_1 - weight_e)  < 0.01: # find l2 norm and check if its smaller 
            wt_1= weight_e # update previous epoch weight
            break
        wt_1= weight_e # update previous epoch weight

    e_out = np.log( 1 + np.exp(-1 * np.array(test_sign_list) * np.matmul(wt_1,test_set.T)))  # find e out using test set with weight
    # np.dot(weight_e.reshape(1,-1), dataset.T)
    eout_list.append(np.average(e_out)) # add e out to list 
    epoch_list.append(e_c) # add epoch counter for that iteration
    
np.average(np.array(eout_list)) # answer is close to 0.1 so D

Wall time: 33.1 s


0.0990123109645455

### Question 9

In [13]:
np.average(np.array(epoch_list)) # answer is close to 350 so A

347.27

## Regularization with weight decay

In [14]:
in_dta = np.loadtxt("in.dta.txt") # read data
out_dta = np.loadtxt("out.dta.txt")
in_data, in_label = in_dta[:,:2], in_dta[:,2] # parse the data relativly
out_data, out_label = out_dta[:,:2], out_dta[:,2] 


In [15]:
def nonlinear_transform(dataset): # apply related nonlinear transform to given dataset
    x1_square = (dataset[:,1]**2).reshape(len(dataset),1) # taking square of x1 column
    x2_square = (dataset[:,2]**2).reshape(len(dataset),1) # taking square of x2 column
    x1_x2 = (dataset[:, 1]* dataset[:, 2]).reshape(len(dataset),1) # multipliying x1 and c2 element wise 
    x1x2_abs = (np.absolute(dataset[:, 1]- dataset[:, 2])).reshape(len(dataset),1) # | x1-x2|
    x1x2_add = (np.absolute(dataset[:, 1] + dataset[:, 2])).reshape(len(dataset),1)#|x1+x2|
    dataset = np.concatenate((dataset,x1_square,x2_square,x1_x2,x1x2_abs,x1x2_add), axis=1) # adding created coulmns to dataset 
    return dataset

def linear_regression(dataset, sign_list):# linear regresssion operation 
    # findind  inverse of x, numpy provides builtin function pinv
    x_pseudo = np.linalg.pinv(dataset)
    # finding new vector based on linear regression formula 
    weight = np.matmul(x_pseudo, sign_list) # w = X_inv y
    return weight 

def run_linreg(train_set, train_label,test_set,test_label): # function to run linear regression
    weight = linear_regression(train_set, train_label) # finding weight 
    predict_train = np.sign(np.matmul(weight, train_set.T))  # get prediction from formula matrix multiplication weight with label
    predict_test = np.sign(np.matmul(weight, test_set.T)) # get prediction from formula matrix multiplication weight with label
    return predict_train, predict_test

def find_ein_eout(p_train, p_test): # find ein and eout 
    miss_train = 0
    for i in range(len(p_train)): # counts miss signs
        if int(p_train[i]) != int(in_label[i]):miss_train+=1
    miss_predict = 0
    for i in range(len(p_test)): # counts miss signs
        if int(p_test[i]) != int(out_label[i]):miss_predict+=1
    return miss_train / len(p_train), miss_predict / len(p_test)


nonlin_train = nonlinear_transform(np.insert(in_data,0,1,axis=1)) # transform train data with nonlinear
nonlin_test = nonlinear_transform(np.insert(out_data,0,1,axis=1)) # transform test data

### Question 2 

In [16]:


p_train, p_test = run_linreg(nonlin_train, in_label,nonlin_test, out_label)

find_ein_eout(p_train, p_test) # answers are close to 0.03 and 0.08, A 


(0.02857142857142857, 0.084)

### Question 3 

In [19]:
def linear_regression_with_weight(dataset, sign_list, k):
    # apply linear regression weight function wit weight decay
    inverse = np.linalg.inv( np.dot(dataset, dataset.T) + 10**k * np.identity(len(dataset)) )
    # finding new vector based on linear regression formula 
    tmp = np.matmul(inverse, dataset) 
    weight = np.matmul(tmp.T, sign_list) # final weight with weight
    return weight
def run_linreg_weight(train_set, train_label,test_set,test_label,k):
    weight = linear_regression_with_weight(train_set, train_label,k) # finding weight 
    predict_train = np.sign(np.matmul(weight, train_set.T))  # train set prediction 
    predict_test = np.sign(np.matmul(weight, test_set.T)) # test set prediction
    return predict_train, predict_test

pw_train, pw_test = run_linreg_weight(nonlin_train, in_label,nonlin_test, out_label, -3)
find_ein_eout(pw_train,pw_test) # answer is close to 0.03 and 0.08, D

(0.02857142857142857, 0.08)

### Question 4

In [21]:
pw_train, pw_test = run_linreg_weight(nonlin_train, in_label,nonlin_test, out_label, 3)
find_ein_eout(pw_train,pw_test) # answer is close to 0.4 and 0.4, E

(0.37142857142857144, 0.436)

### Question 5 & 6

In [29]:
pw_train2, pw_test2 = run_linreg_weight(nonlin_train, in_label,nonlin_test, out_label, 2)
pw_train1, pw_test1 = run_linreg_weight(nonlin_train, in_label,nonlin_test, out_label, 1)
pw_train0, pw_test0 = run_linreg_weight(nonlin_train, in_label,nonlin_test, out_label, 0)
pw_train1n, pw_test1n = run_linreg_weight(nonlin_train, in_label,nonlin_test, out_label, -1)
pw_train2n, pw_test2n = run_linreg_weight(nonlin_train, in_label,nonlin_test, out_label, -2)
(find_ein_eout(pw_train2,pw_test2)[1] ,
find_ein_eout(pw_train1,pw_test1)[1],
find_ein_eout(pw_train0,pw_test0)[1],
 find_ein_eout(pw_train1n,pw_test1n)[1],
 find_ein_eout(pw_train2n,pw_test2n)[1],
) # when k=-1, the out of sample error is the smallest, D

(0.228, 0.124, 0.092, 0.056, 0.084)

The smallest is 0.056 so, it is closer to 0.06, Answer for Q6 is B

## Neural Networks

### Question 10


In [30]:
hidden_units = 36
max_w = 0 # to set maximum into this variable 
for unit_l1 in range(2, 34):
    unit_l2 = hidden_units - unit_l1 # number of layer 2 unit
    num_w = 0 # first intialize number weight to 0
    num_w += ( unit_l1 - 1) * 10 #  input layer to layer 1
    num_w +=  ( unit_l2 - 1 ) * unit_l1 # layer 1 to layer 2
    num_w += 1 * unit_l2 # layer 2 to output
    if num_w > max_w:max_w = num_w
    print( "total weights", num_w)
print( "Maximum is", max_w ) # answer is 510, E

total weights 110
total weights 149
total weights 186
total weights 221
total weights 254
total weights 285
total weights 314
total weights 341
total weights 366
total weights 389
total weights 410
total weights 429
total weights 446
total weights 461
total weights 474
total weights 485
total weights 494
total weights 501
total weights 506
total weights 509
total weights 510
total weights 509
total weights 506
total weights 501
total weights 494
total weights 485
total weights 474
total weights 461
total weights 446
total weights 429
total weights 410
total weights 389
Maximum is 510
